# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#loading the data from city.csv into the data frame
city_data=pd.read_csv("../output_data/cities.csv")
city_data

,City Id,City,Country,Date,Latitude,Longitude,Temperature,Max Temperature,Humidity,Wind Speed,Cloudiness
0,1,ushuaia,AR,1586669773,-54.80,-68.30,39.20,39.20,64,9.17,40
1,2,la joya,MX,1586669773,18.67,-99.47,74.82,74.82,20,2.89,0
2,3,bredasdorp,ZA,1586669773,-34.53,20.04,66.20,66.20,68,25.28,71
3,4,new norfolk,AU,1586669774,-42.78,147.06,57.63,60.80,59,14.99,75
4,5,castro,BR,1586669774,-24.79,-50.01,55.69,55.69,69,5.48,61
...,...,...,...,...,...,...,...,...,...,...,...
557,558,namatanai,PG,1586669615,-3.67,152.43,84.06,84.06,72,6.17,98
558,559,hazleton,US,1586669884,40.96,-75.97,31.66,37.40,74,2.04,1
559,560,kaduna,NG,1586669884,10.52,7.44,76.01,76.01,24,2.33,100
560,561,sibolga,ID,1586669884,1.74,98.78,87.80,87.80,67,4.43,79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configuring gmaps
gmaps.configure(api_key=gkey)
city_humidity=city_data['Humidity']
city_coordinates = city_data[['Latitude', 'Longitude']]

humidity_map = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
humidity_map=gmaps.figure()

#adding heatmap layer
humidity_layer = gmaps.heatmap_layer(city_coordinates, weights=city_humidity)

humidity_map.add_layer(humidity_layer)

humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Narrowing down the cities to fit weather conditions
city_data_ideal=city_data.loc[(city_data['Temperature']>70) & (city_data['Temperature']<80)& (city_data['Wind Speed']<10)
                               &(city_data['Cloudiness']==0)]
                           
                           
city_data_ideal


,City Id,City,Country,Date,Latitude,Longitude,Temperature,Max Temperature,Humidity,Wind Speed,Cloudiness
1,2,la joya,MX,1586669773,18.67,-99.47,74.82,74.82,20,2.89,0
215,216,maxixe,MZ,1586669815,-23.86,35.35,75.20,75.20,78,5.61,0
312,313,tessalit,ML,1586669833,20.20,1.01,76.78,76.78,15,5.88,0
334,335,port macquarie,AU,1586669838,-31.43,152.92,71.67,72.00,35,4.70,0
400,401,macia,MZ,1586669852,-25.03,33.10,72.72,72.72,71,7.72,0
419,420,taoudenni,ML,1586669856,22.68,-3.98,75.63,75.63,18,3.44,0
439,440,aranos,NaN,1586669860,-24.13,19.12,70.63,70.63,32,5.23,0
460,461,inhambane,MZ,1586669864,-23.86,35.38,75.20,75.20,78,5.68,0
491,492,gunjur,GM,1586669871,13.20,-16.73,71.60,71.60,88,4.70,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Hotel Data Frame
hotel_df=city_data_ideal[['Latitude','Longitude','City', 'Country']].copy()
hotel_df['Hotel Name']=""
hotel_df

,Latitude,Longitude,City,Country,Hotel Name
1,18.67,-99.47,la joya,MX,
215,-23.86,35.35,maxixe,MZ,
312,20.20,1.01,tessalit,ML,
334,-31.43,152.92,port macquarie,AU,
400,-25.03,33.10,macia,MZ,
419,22.68,-3.98,taoudenni,ML,
439,-24.13,19.12,aranos,NaN,
460,-23.86,35.38,inhambane,MZ,
491,13.20,-16.73,gunjur,GM,


In [19]:
import json

# # base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="

#Hotel search for the desired city
for index, row in hotel_df.iterrows():
    lat=row["Latitude"]
    lon=row["Longitude"]
    search_url= base_url + str(lat) +","+str(lon)+"&radius=5000"+"&type=lodging"+"&keyword=hotel"+"&key="+ gkey
    response = requests.get(search_url).json()
        try:
        hotel_df.loc[index,'Hotel Name'] = response["results"][0]['name']
    except IndexError:
        hotel_df.loc[index,'Hotel Name']= ""
        print("No hotel found within 5000 meter radius")
hotel_df

No hotel found within 5000 meter radius
No hotel found within 5000 meter radius
No hotel found within 5000 meter radius


,Latitude,Longitude,City,Country,Hotel Name
1,18.67,-99.47,la joya,MX,
215,-23.86,35.35,maxixe,MZ,Hotel Casa Do Capitão
312,20.20,1.01,tessalit,ML,
334,-31.43,152.92,port macquarie,AU,Sails Port Macquarie by Rydges
400,-25.03,33.10,macia,MZ,Hotel Taskin Macia
419,22.68,-3.98,taoudenni,ML,
439,-24.13,19.12,aranos,NaN,Aranos Hotel
460,-23.86,35.38,inhambane,MZ,Hotel Casa Do Capitão
491,13.20,-16.73,gunjur,GM,"Footsteps Eco Lodge, The Gambia"


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [23]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))